Примеры основных базовых методов на SQL postgres на базе проката DVD фильмов

Диаграмма "сущность-связь" (ERD, от англ. Entity-Relationship Diagram) по ссылке
[схема](https://www.postgresqltutorial.com/postgresql-getting-started/postgresql-sample-database/)


In [1]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa

In [2]:
# устанавливаем параметры
db_config = {'user': 'postgres', # имя пользователя
             'pwd': 'mysecretpassword', # пароль
             'host': 'localhost',
             'port': 5432, # порт подключения
             'db': 'dvdrental' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={})

In [3]:
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    '''Открываем соединение, получаем данные из sql, закрываем соединение'''
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con = con)

In [4]:
# Установка параметров отображения
pd.set_option('display.max_colwidth', None)

#### Запросы

In [15]:
# формируем запрос и выводим данные
query = '''
SELECT *
FROM payment
limit 5
'''
get_sql_data(query)

,payment_id,customer_id,staff_id,rental_id,amount,payment_date
0,17503,341,2,1520,7.99,2007-02-15 22:25:46.996577
1,17504,341,1,1778,1.99,2007-02-16 17:23:14.996577
2,17505,341,1,1849,7.99,2007-02-16 22:41:45.996577
3,17506,341,2,2829,2.99,2007-02-19 19:39:56.996577
4,17507,341,2,3130,7.99,2007-02-20 17:31:48.996577


Сколько принес денег каждый покупатель?

In [11]:
# формируем запрос и выводим данные
query = '''
SELECT customer_id, sum(amount)
FROM payment
GROUP BY customer_id
limit 5

'''
get_sql_data(query)

,customer_id,sum
0,1,114.70
1,2,123.74
2,3,130.76
3,4,81.78
4,5,134.65


Сколько транзакций в день проходит?

In [22]:
# формируем запрос и выводим данные
query = '''
SELECT  DATE_TRUNC('day', payment_date) as date, sum(amount) as amount
FROM payment
GROUP BY DATE_TRUNC('day', payment_date)
order by date
limit 5

'''
get_sql_data(query)

,date,amount
0,2007-02-14,116.73
1,2007-02-15,1188.92
2,2007-02-16,1154.18
3,2007-02-17,1188.17
4,2007-02-18,1275.98


Нужны электронные адреса тех кто платил 14 февраля?

In [25]:
# формируем запрос и выводим данные
query = '''
SELECT  distinct email
FROM payment p
join customer c on p.customer_id = c.customer_id
where DATE_TRUNC('day', payment_date) = '2007-02-14'
limit 5

'''
get_sql_data(query)

,email
0,alma.austin@sakilacustomer.org
1,amber.dixon@sakilacustomer.org
2,audrey.ray@sakilacustomer.org
3,catherine.campbell@sakilacustomer.org
4,cecil.vines@sakilacustomer.org
